In [ ]:
import pandas as pd
import openpyxl
import re
import os

# Function to extract hyperlinks from Excel
def extract_hyperlinks(excel_file_path):
    wb = openpyxl.load_workbook(excel_file_path)
    sheet = wb.active
    hyperlinks = []

    # Loop through the rows in the first column to find hyperlinks
    for row in sheet.iter_rows(min_row=1, max_col=1, values_only=False):
        cell = row[0]
        if cell.hyperlink:
            hyperlinks.append(cell.hyperlink.target)
        else:
            hyperlinks.append(None)
    
    return hyperlinks

# Function to extract 5-digit zip codes from the "Name:" column
def extract_zip_code(address):
    if pd.notna(address):
        match = re.search(r'\b\d{5}\b', address)
        if match:
            return match.group(0)
    return None

# Directory paths to CSV and Excel files for candidates
csv_directory = 'Add your directory path here'
excel_directory = 'Add your directory path here'


# List to store all candidate dataframes
all_cleaned_dataframes = []

# Iterate over each candidate's CSV and Excel files
for csv_file in os.listdir(csv_directory):
    if csv_file.endswith('.csv'):
        # Load the CSV file into a DataFrame
        file_path_csv = os.path.join(csv_directory, csv_file)
        df_candidate = pd.read_csv(file_path_csv)

        # Extract zip codes from the "Name:" column (adjust column name if different)
        zip_codes_full = df_candidate['Name:'].apply(extract_zip_code)

        # Assign zip codes to the first row of each block by shifting the zip codes up by 2 rows (from the second row)
        df_candidate['ZipCode'] = zip_codes_full.shift(-2)

        # Clean the data by retaining only the first row in each block (where 'Report Id:' is not NaN)
        clean_df = df_candidate[df_candidate['Report Id:'].notna()].copy()

        # Find the corresponding Excel file for hyperlinks
        excel_file_name = csv_file.replace('.csv', '.xlsx')
        excel_file_path = os.path.join(excel_directory, excel_file_name)

        # Extract hyperlinks from the Excel file
        hyperlinks = extract_hyperlinks(excel_file_path)

        # Add hyperlinks to the cleaned DataFrame
        clean_hyperlinks = []
        link_idx = 0
        for i in range(len(clean_df)):
            while link_idx < len(hyperlinks) and hyperlinks[link_idx] is None:
                link_idx += 1
            if link_idx < len(hyperlinks):
                clean_hyperlinks.append(hyperlinks[link_idx])
                link_idx += 1
            else:
                clean_hyperlinks.append(None)

        # Add the 'ReportLink' column to the DataFrame
        clean_df['ReportLink'] = clean_hyperlinks

        # Append the cleaned DataFrame for this candidate to the list
        all_cleaned_dataframes.append(clean_df)

        print(f"Processed and aggregated: {csv_file}")

# Concatenate all candidate DataFrames into a single DataFrame
final_combined_df = pd.concat(all_cleaned_dataframes, ignore_index=True)

# Save the final combined DataFrame with hyperlinks and zip codes as a new CSV
final_combined_df.to_csv('aggregate_005.csv', index=False)

